In [9]:
# Ex.1
# Performs matrix multiplication of two 2x2 matrices
def mult_2x2(x,y):
    a = x[0][0]*y[0][0] + x[0][1]*y[1][0]
    b = x[0][0]*y[0][1] + x[0][1]*y[1][1]
    c = x[1][0]*y[0][0] + x[1][1]*y[1][0]
    d = x[1][0]*y[0][1] + x[1][1]*y[1][1]
    return [[a,b],[c,d]]
    
# Performs the final step of extracting the desired Fibonacci numbers 
def final_form(M, pk1, pk2):
    a = M[0][0]*pk1 + M[0][1]*pk2
    b = M[1][0]*pk1 + M[1][1]*pk2
    return [a,b]
    
# Performs fast matrix exponentiation
def matrix_power(x,n):
    if (n == 1):
        return x
    if ( n % 2 == 0 ):
        return matrix_power(mult_2x2(x,x),n//2)
    return mult_2x2(x, matrix_power(mult_2x2(x, x), n//2))

pk1 = 616008725853825692345385361476108686236820306437245
pk2 = 672776316565322588692456541214379727209632393531548
p = 783327858451078922569517728615115361780490687182179
sk = 1541239189119920869444739

R = IntegerModRing(p)
M = Matrix(R, [[1,1], [1,0]])
An = Matrix(R, matrix_power(M, sk))
b = vector(R, final_form(An, pk1, pk2))
print(b[0])


368527336760012409385648278024182667570755740958494


In [10]:
# Ex.2
import math
import base64

Q2_I=[7, 13, 9, 6, 2, 4, 14, 5, 3, 1, 12, 11, 8, 10]
Q2_II=[7, 6, 5, 1, 4, 3, 2]
Q2_ct="ISoFVckmGsFlGMW0d3kWaYBWGg5cbpas5sHoBNayvH5lZG5GZ53XYI0XWvybZybyNFGcaNIzlCZInItXIWzB5aBWW0ZJXnaiJgWYNRZ0s2dQZGFW"


def decryptCipher(cipher, key, N): 
    pt = "" 
    
    # tracking indices for the key and the cipher
    key_idx = 0
    cipher_idx = 0
  
    # calculate matrix dimensions and initialize empty matrix
    # to store decrypted message
    cipher_length = float(len(cipher)) 
    num_cols = len(key)    
    num_rows = int(math.ceil(cipher_length / num_cols)) 
    decrypted = [] 
    for _ in range(num_rows): 
        decrypted += [[None] * num_cols]
    
    # sort the key
    sorted_key = sorted(list(key))
    cipher_list = list(cipher)

    # arrange the matrix column wise according  
    # to permutation order by adding into new matrix 
    for _ in range(num_cols): 
        # get current column index based on the key
        curr_idx = key.index(sorted_key[key_idx]) 
        # take offset into account (add offset mod number of columns)
        curr_idx = (curr_idx+N) % num_cols
  
        # populate decrypted matrix
        for j in range(num_rows): 
            decrypted[j][curr_idx] = cipher_list[cipher_idx] 
            cipher_idx += 1
        key_idx += 1
  
    # convert decrypted matrix into a string  
    pt = ''.join(sum(decrypted, [])) 
    
    # strip trailing space (e.g. padding)
    return pt.rstrip() 


# reverse the ascii + base64 encoding
def decodeCipher(base64_msg):
    base64_bytes = base64_msg.encode("ascii") 

    message_bytes = base64.b64decode(base64_bytes) 
    message_string = message_bytes.decode("ascii") 
  
    print("Decoded: " + message_string) 
    

for N in range(20):
    try:
        msg1 = decryptCipher(Q2_ct, Q2_II, N)
        msg2 = decryptCipher(msg1, Q2_I, N)
        print(decodeCipher(msg2))
        print("offset = " + str(N))
        break
    except:
        print("incorrect offset")

incorrect offset
incorrect offset
incorrect offset
Decoded: And slanderous spring from pestilential breath Each female basilisk with forky sting
None
offset = 3


In [11]:
# Ex.3
import math
cipher_alphabet = ["A", "D", "F", "G", "V", "X"]
my_Q3a_y="FXDVAGFGGGVVGGFDDFVADVFVFGFGVAFDFVA"  # cihpertext
my_Q3a_S='I9K1TB4YS3HMWU5OG0VNE78XDZARLPCF62JQ' # substitution
my_Q3a_c="A"                                    # padding character
my_Q3a_s=(5, 2, 0, 6, 3, 4, 1)                  # permutation
my_Q3a_n=16                                     # plaintext length

def invert_permutation(perm):
    inverse = [0] * len(perm)
    for i, p in enumerate(perm):
        inverse[p] = i
    return inverse

# Problem 3a
def my_simple_decrypt(Q3a_y, Q3a_S, Q3a_c, Q3a_s, Q3a_n):
    # compute clumns and rows of the cipher text matrix
    # there are as many columns in the cipher text as the length of the permutation
    columns = len(Q3a_s)
    rows = int(len(Q3a_y) / columns)

    # extract the columns from the cipher text
    permuted_cipher_columns = [Q3a_y[x:x+rows] for x in range(0,len(Q3a_y),rows)]
    # invert the permutation
    inverse_permutation = invert_permutation(Q3a_s)
    # undo the permutation on the cipher columns
    unpermuted_cipher_columns = [permuted_cipher_columns[i] for i in inverse_permutation]

    # build the cipher rows and flatten the cipher into one string
    zeroth_row = ""
    first_row = ""
    second_row = ""
    third_row = ""
    fourth_row = ""
    for c in unpermuted_cipher_columns:
        zeroth_row += c[0]
        first_row += c[1]
        second_row += c[2]
        third_row += c[3]
        fourth_row += c[4]
    flattened_cipher = zeroth_row + first_row + second_row + third_row + fourth_row

    # inflate S into 2D form
    S_2d = [Q3a_S[x:x+6] for x in range(0,len(Q3a_S),6)]
    counter = 0
    row_id = 0
    col_id = 0
    plaintext = ""
    # iterate over the flattened cipher text and take pairs of consecutive characters
    # and lookup the character's index in the cipher_alphabet
    # the first character is the row_id of S, the second is the col_id
    for c in flattened_cipher:
        if (counter%2 == 0):
            row_id = cipher_alphabet.index(c)
        if (counter%2 == 1):
            col_id = cipher_alphabet.index(c)
            plaintext += S_2d[row_id][col_id]
        counter += 1
    
    # we know the plaintext length so take only the first that many characters of decrypted message
    return plaintext[:Q3a_n]

print(my_simple_decrypt(my_Q3a_y, my_Q3a_S, my_Q3a_c, my_Q3a_s, my_Q3a_n))


ANEHSAFEENORLLWE


In [12]:
Q3b_x="WELCOME TO THE BONFIRE, UNKINDLED ONE. I AM A FIRE KEEPER. I TEND TO THE FLAME, AND TEND TO THEE. THE LORDS HAVE LEFT THEIR THRONES, AND MUST BE DELIVER'D TO THEM. TO THIS END, I AM AT THY SIDE."
Q3b_S='LJY6R5OFGHEA0BK7TSV31CID9ZMPUN2QXW84'
Q3b_c="V"
Q3b_s=(3, 1, 11, 6, 2, 10, 7, 12, 9, 8, 4, 5, 0)

# Problem 3b
def my_extended_encrypt(Q3b_x, Q3b_S, Q3b_c, Q3b_s):
    # compute clumns and rows of the cipher text matrix
    # there are as many columns in the cipher text as the length of the permutation 
    S_2d = [Q3b_S[x:x+6] for x in range(0,len(Q3b_S),6)]
    cipher = ""
    row_id = 0
    col_id = 0
    for c in Q3b_x:
        if (c.isnumeric()) or (c.isalpha() and c.isupper()):
            # if it is an alpha-numberical, perform a substitution
            for row in S_2d:
                if c in row:
                    row_id = S_2d.index(row)
                    col_id = row.index(c)
                    break
            cipher += cipher_alphabet[row_id]
            cipher += cipher_alphabet[col_id]
        else:
            # if it is a special character, x -> (x,x)
            cipher += c
            cipher += c
    
    # add padding
    columns = len(Q3b_s)
    for i in range(columns - len(cipher)%columns):
        cipher += Q3b_c
    rows = math.ceil(len(cipher) / columns)

    # convert cipher into columns
    unpermuted_cipher_columns = [cipher[x:x+columns] for x in range(0,len(cipher),columns)]
    
    # perform the permutation on the cipher columns
    permuted_cipher_columns = []
    for r in unpermuted_cipher_columns:
        r = [r[i] for i in Q3b_s]
        concatenated_r = ""
        for c in r:
            concatenated_r += c
        permuted_cipher_columns.append(concatenated_r)
    
    # build the cipher text from the permuted cipher columns
    cipher_text = ""
    for i in range(columns):
        for r in permuted_cipher_columns:
            cipher_text += r[i]

    return cipher_text

my_extended_encrypt(Q3b_x, Q3b_S, Q3b_c, Q3b_s)

"VF DVD GV VD VVDX VD,V A FXGVGG VAF VDVG  ,D FVD FXVVDA V XFFDDVV XF.GGD D.DXFVDXFVFFDX FVGAD,GXVVDV AVXDV ADVDXD'DAD FXD  VG  DDVF  VFVGVFV,F V  F FXVVDVD.D .XDF V.ADDAXG G VVGFA.G A XDFDVD DX V  DVDXFVGG VDVDDGGXG  F XDVFA AGV V DGV.G,  VAFXVA.  VVVVXFVAGDVVX A VFXVG.D V AV VAVFXG D DVGAVVAXD VXDVAVFVVADVV DDDXDGF DG VGVFV V VADD,XVXAGFAAXGGDXAGA FX'VD   GXVDVFXG D  V,V  AA  F DAD.G,D "

In [13]:
my_Q3c_x0="OBI-WAN: IT'S OVER, ANAKIN! I HAVE THE HIGH GROUND! ANAKIN SKYWALKER: YOU UNDERESTIMATE MY POWER! OBI-WAN: DON'T TRY IT."
my_Q3c_y0="DV-FF:DFX XA, FXDA DFXA FAFDF VAFX XXXF VFXDAA FF FXVAFDXF D VGA! ADGX: DAF VFDFADGX: F' AAA XXXF!D XA FA DX XDGA!XAVD DAFGXV:AA GAAADDDFAAFFAAA AD-XA XF' FA F.AD-XA D'DDAV,XAVD!  XXAFX XGXGAFA! FXDAXXGXVA: DGFAAAXFAXA A DFV!DV-FF:AA'FFA D."
my_Q3c_y1="AD FF XA XXAFX FXXAFDAAXFXFDGGDDFX FXA DFXFXF DF D'DFA XXFVFFX GADGAGXFAG DG D'DX DF DGAA.AXFXFDGGDDGX XAXV DA A FA DF, D DFVAG FXXAGDA A DGA XAFX AVV F DGDAAA FAADDVGADXX D VAFAGDA.FAFX DVXX FAGAGADAFX XVXXDA FX X VFD AAAVAF FADAX A XAA AAF VA XDA. FAAXV DX A FA DAA X  VFXXFFAAXA ADDFADXAA DADXV D A FAFGXD X VVXA D DFVAG FXFX AGFFXFXFAGX XVXA G XDGADA XX DFV,GXVXAAFFXDA,DAVAVD, A DX GGAFGXFGF X FFXF DDXVVAFDA XAAFDA X XAF FAAFD XFAFFVAVDGA XA F DDXDAV DDAV.FAVAGXXXA FXVDGAA AXF,AG AF DDAD.DXAA AAAFAV FAFAGAA G XAF GXFAX FAGDA?D FAGX AF.DFX DFX FAA FAXAD DGA AD FF.DFX  DFXGAAA. XAF GXFAX XDX XA DVXDAFX DFX D VGAGG XA D DD X VFD GA FAGAVAFADAGGFV FA DXVXDAXA F VAXA DA. A XXXGF  XDFAXA G FAAXV DX FXFFAVAGXAAF AA FA FA X VVA XDG ADDAFFX FA XA DAADAFX AVV D XFXFFA D XA XDDFDX DD VFDAA F V GFXFAG.FAAAXD D VGAGG XA FADAA FDG A XDXAXD A FX DDFXFD VGA, FDF XAFGGD, G DGXA X XA.FADAFAFAA,FAFXGX XX FVAADV AAAFXFXFAVAG,FXF DDXVVAFDA DDAXFD D XX GAF.DAFD. A DGA XA AFAX VA XXF, FFFA XADGAVXA DGAAV AX FA VXAX A XVF VXXAD FA DD?  FDGFFFA. F' AF  DDA FA AX FFD FGDAA. F' XXDF AXFX XVF VXXAD FX  XVXGAA G FAXDF,XAFAAAFDXA D FXAFAVAGXFD FA DAA D FAFAVAFX DADFDVDFDFAVAXF DGA X XA FV XVAGDAX A FA XA DAAFXF A DGA XA XAVFX AADFAVXAXXVFFGAA ADG.FX XVXXDA G FAGAVADDX XFGX F DFFXVGDDA DAAVAXDAAFAAAFAXFVX. A AVAAXAFAAAFDXA FA FD FDA X FX GADXDAGX DXDG DDFAAA FDV AAAFXGF A VFAA,FAAD. FAVFFXAD, A XGFFFD XFAFFVAAAVFFDG A FA, FA XX FVAADV XGDA DDDAFD DAA. VADV X VFD DXADFAA ADDAAFX VF DFFDXAG."
my_Q3c_S='E\x00\x002\x00\x00\x00\x009584\x006\x00U\x00\x00F\x00\x0071\x00RZQ0\x00\x00\x00\x00XJ3A'
my_Q3c_c="V"
my_Q3c_H="2039f0a25d6702cf9a8e942faddd0362"

import string
P = string.punctuation + string.whitespace

import random

# Problem 3c
def my_incomplete_decrypt(Q3a_y, Q3a_S, Q3a_c, Q3c_H, Q3c_s):
    # compute clumns and rows of the cipher text matrix
    # there are as many columns in the cipher text as the length of the permutation
    columns = len(Q3c_s)
    rows = int(len(Q3a_y) / columns)
    # inflate S into 2D form
    S_2d = [Q3a_S[x:x+6] for x in range(0,len(Q3a_S),6)]

    # extract the columns from the cipher text
    permuted_cipher_columns = [Q3a_y[x:x+rows] for x in range(0,len(Q3a_y),rows)]
    # invert the permutation
    inverse_permutation = invert_permutation(Q3c_s)
    # undo the permutation on the cipher columns
    unpermuted_cipher_columns = [permuted_cipher_columns[i] for i in inverse_permutation]

    # build the cipher rows and flatten the cipher into one string
    flattened_cipher = ""
    for r in range(rows):
        for col in unpermuted_cipher_columns:
            flattened_cipher += col[r]

    counter = 0
    row_id = 0
    col_id = 0
    plaintext = ""
    # iterate over the flattened cipher text and take pairs of consecutive characters
    # and lookup the character's index in the cipher_alphabet
    # the first character is the row_id of S, the second is the col_id
    for c in flattened_cipher:
        if (counter%2 == 0):
            if c in P:
                plaintext += c
            else:
                row_id = cipher_alphabet.index(c)
        elif (counter%2 == 1):
            if c not in P and c != "*":
                col_id = cipher_alphabet.index(c)
                symbol = S_2d[row_id][col_id]
                if symbol == '\x00':
                    symbol = "*"
                plaintext += symbol
        counter += 1
    
    # we know the plaintext length so take only the first that many characters of decrypted message
    return plaintext

import hashlib
import base64

# Recovered the permutation by hand on paper while trying out the enryption/decryption scheme
Q3c_s = (0, 2, 1)

# manually recovered the full S-matrix by trials using x0 and y0
my_trial_Q3c_S='EMY2BDOI9584N6TUPHFLW71GRZQ0CKVSXJ3A'

pt = my_incomplete_decrypt(my_Q3c_y1, my_trial_Q3c_S, my_Q3c_c, my_Q3c_H, Q3c_s)
print(pt)
def get_hex_digest(pt):
    s = pt.encode()
    r = base64.b64encode(s) 
    H = hashlib.md5(r).hexdigest() # '2039f0a25d6702cf9a8e942faddd0362'
    return H

print(get_hex_digest(pt))

DID YOU EVER HEAR THE TRAGEDY OF DARTH PLAGUEIS THE WISE? I THOUGHT NOT. IT'S NOT A STORY THE JEDI WOULD TELL YOU. IT'S A SITH LEGEND. DARTH PLAGUEIS WAS A DARK LORD OF THE SITH, SO POWERFUL AND SO WISE HE COULD USE THE FORCE TO INFLUENCE THE MIDICHLORIANS TO CREATE LIFE. HE HAD SUCH A KNOWLEDGE OF THE DARK SIDE THAT HE COULD EVEN KEEP THE ONES HE CARED ABOUT FROM DYING. THE DARK SIDE OF THE FORCE IS A PATHWAY TO MANY ABILITIES SOME CONSIDER TO BE UNNATURAL. HE BECAME SO POWERFUL AND THE ONLY THING HE WAS AFRAID OF WAS LOSING HIS POWER, WHICH EVENTUALLY, OF COURSE, HE DID. UNFORTUNATELY, HE TAUGHT HIS APPRENTICE EVERYTHING HE KNEW, THEN HIS APPRENTICE KILLED HIM IN HIS SLEEP. IRONIC. HE COULD SAVE OTHERS FROM DEATH, BUT NOT HIMSELF.
2039f0a25d6702cf9a8e942faddd0362


In [16]:
Q3d_x0="OBI-WAN: IT'S OVER, ANAKIN! I HAVE THE HIGH GROUND! ANAKIN SKYWALKER: YOU UNDERESTIMATE MY POWER! OBI-WAN: DON'T TRY IT."
Q3d_y0="F'GA AGDGVAFAFVG-DA.XAAFA AFF XVDAFAA  AXFGXA ADX GGXAFGA  FF:  GXG XXGF GFA XGGA FF!A G!AX:DVAFFG' FGV,ADDDGFAADGF !XFG-AVG GGF GX GF XF:FAX'FX FXVGDXDGFGF-XF.A DX!V X!AX:VDGDDG' DGD,AGGGGXXGVFX !XDFV:  GFA DAVG AAG FGV-AAG ADG GF GA XV:AA"
Q3d_y1="FDGGF AF AF VF AFAXXGAAX.FD.A .AD. VAXFFFAF.XG.V .AG. GGXGDDGFAGG  AXGG G',.AA GVX GADA. GAVV XAGGF FFDVA GFAVVADF  D  GF FX..D F. GXDVFGX FX GX DX ADAXGG XGG FAGF. FADD XFFGD FFGDA GFFDGFVXDDVGFAVA  ADVA A',.  V  VX AF..V A. XX"
initial_Q3d_S='\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0040\x00P5\x00\x00Q\x00J\x00B\x0082Z7\x00\x006W'
Q3d_c="X"
Q3d_s=(10, 8, 9, 3, 4, 0, 7, 11, 5, 1, 2, 6)
Q3d_S='I*TEV\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0040\x00P5\x00\x00Q\x00J\x00B\x0082Z7\x00\x006W'
Q3d_H="cc84f04a547288918732dd100954b5ee"

# Problem 3d
def reconstruct_s(Q3d_S):
    alphabet = ["A","D","E","G","H","I","K","L","M","N","O","P","R","S","T","U","V","Y"]
    initial_matching = 0
    incomplete_pt = my_incomplete_decrypt(Q3d_y0, Q3d_S, Q3d_c, Q3d_H, Q3d_s)
    # count how many matching characters does the x0 and incomplete decrypted y0 have
    for i in range(len(incomplete_pt)):
        if incomplete_pt[i] == Q3d_x0[i]:
            initial_matching += 1
    
    # find the first unknown character's index in S
    first = Q3d_S.find('\x00')
    # try to put each of the remaining unknown letters in that position
    for a in alphabet:
        temp_S = ""
        for idx in range(len(Q3d_S)):
            if idx != first:
                temp_S += Q3d_S[idx]
            else:
                temp_S += a
        # decrypt with the updated S and count the new number of matching characters
        incomplete_pt = my_incomplete_decrypt(Q3d_y0, temp_S, Q3d_c, Q3d_H, Q3d_s)
        count_known = 0
        count_matching = 0
        for i in range(len(incomplete_pt)):
            if incomplete_pt[i] != "*":
                count_known += 1
                if incomplete_pt[i] == Q3d_x0[i]:
                    count_matching += 1
        # if we have more matching characters than originally, then we found a new letter in S
        if count_known == count_matching and count_matching > initial_matching:
            print("index " + str(first) + " :: letter = " + a)
            break
            
# run reconstruct_s() with the initial_Q3d_S
# each run - if an index and letter are printed, substitute the corresponding character in improved_Q3d_S with the letter
#            if no index & letter are printed, substitute the corresponding character in improved_Q3d_S with '*'
initial_Q3d_S='\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0040\x00P5\x00\x00Q\x00J\x00B\x0082Z7\x00\x006W'

# partially reconstructed Q3d_S after a few iterations of reconstruct_s()
improved_Q3d_S='I*TEVK***HUD*OM40AP5RNQGJSBY82Z7*L6W'

reconstruct_s(improved_Q3d_S)

# Run the incomplete decryption function from 3c and bruteforce/manually improve S until the full sentence is printed
# Or guess the rest of the unknown letters
incomplete_pt = my_incomplete_decrypt(Q3d_y1, improved_Q3d_S, Q3d_c, Q3d_H, Q3d_s)
print(incomplete_pt)

# Final reconstructed Q3d_S
final_Q3d_S='I*TEVK*C*HUD*OM40AP5RNQGJSBY82Z7FL6W'
Q3d_H="cc84f04a547288918732dd100954b5ee"

# Remove characters from the end one by one until the hexdigest matches
pt = my_incomplete_decrypt(Q3d_y1, final_Q3d_S, Q3d_c, Q3d_H, Q3d_s)
for i in range(len(pt)):
    hex_digest = get_hex_digest(pt[:len(pt)-i])
    if hex_digest == Q3d_H:
        print(pt[:len(pt) - i])
        break

THE *OR*E RUNS STRONG IN MY *AMILY. MY *ATHER HAS IT. I HAVE IT. AND... MY SISTER HAS IT. YES. IT'S YOU, LEIA.WWWW
THE FORCE RUNS STRONG IN MY FAMILY. MY FATHER HAS IT. I HAVE IT. AND... MY SISTER HAS IT. YES. IT'S YOU, LEIA.
